In [ ]:
import os
import numpy as np
import warnings
from astropy.coordinates import Angle
from astropy.io import fits

from astropy.modeling import models, fitting
import glob
from barycorrpy import get_BC_vel
from scipy import interpolate
import matplotlib.pyplot as plt
import json
import time
import configparser
import csv
from astropy.time import Time
%matplotlib inline

In [ ]:
from AlgorithmDev import RadialVelocity
FIT_G = fitting.LevMarLSQFitter()
LIGHT_SPEED = 299792.458   # light speed in km/s
TEST_DIR = '../test_data/'

In [ ]:
def plot_gaussian_on_curve(g_curve, curve_x, curve_y, order=None, title=None, ref_curve=None, ref_gaussian=None):
    plt.figure(figsize=(10,10))
    
    label_curve = "rv on order " + str(order) if order is not None else 'all orders '
    plt.plot(curve_x, curve_y, 'ko', label=label_curve)
    if ref_curve is not None:
        plt.plot(curve_x, ref_curve, 'mo', label="neid_ccf"  )
    ref_label = '/'+str("{0:.6f}".format(ref_gaussian.mean.value)) if ref_gaussian is not None else ''
    plt.plot(curve_x, g_curve(curve_x), label='mean:'+str("{0:.6f}".format(g_curve.mean.value))+ ref_label)
                                                      
    plt.legend(loc="lower right", prop={'size': 12})
    plt.title(title) if title is not None else None
    plt.show()

In [ ]:
def write_ccf_to_csv(ccf, csvfile):
    with open(csvfile, mode='w') as result_file:
        result_writer = csv.writer(result_file)
        for i in range(0, np.shape(ccf)[0]):
            row_data = list()
            row_data.append(i)
            for val in ccf[i, :]:
                row_data.append(val)
            result_writer.writerow(row_data)

In [ ]:
def compare_two_csv(file1, file2):
    file1_rows = list()
    file2_rows = list()
    
    with open(file1) as csv1:
        csvReader1 = csv.reader(csv1)
        for row in csvReader1:
            file1_rows.append(row)
    with open(file2) as csv2:
        csvReader2 = csv.reader(csv2)
        for row in csvReader2:
            file2_rows.append(row)
   
    f1 = np.array(file1_rows, dtype=float)
    f2 = np.array(file2_rows, dtype=float)
    
    diff_index = np.where((f1-f2) != 0.0)    
    
    diff_at = list(zip(list(diff_index[0]), list(diff_index[1])))

    return diff_at

In [ ]:
import math
def plot_velocity_time(ref_date, rv_info, title=None, label=[], color=None, savefig=None):
    #print(ref_date)
    #import pdb;pdb.set_trace()
    ref_time = Time(ref_date, format='isot', scale='utc')  
    ref_jd = ref_time.jd
    
    total_rv = len(rv_info)
    jd_list = np.array([rv_info[i][1] for i in range(total_rv)])   
    
    rv_del_hr = (jd_list-ref_jd)*24.0
    rv_ref = np.array([rv_info[i][2] for i in range(total_rv)])
    rv_developed = np.array([rv_info[i][3] for i in range(total_rv)])
   
    rv_ref_offset = (rv_ref - np.mean(rv_ref))*1000
    rv_developed_offset = (rv_developed - np.mean(rv_developed)) * 1000
   
    rv_ref_sigma = np.std(rv_ref_offset)
    rv_developed_sigma = np.std(rv_developed_offset)
    
    plt.figure(figsize=(10,12))  
    s = 100
    
    if color is None:
        color = ['cyan', 'yellow']
    plt.scatter(rv_del_hr, rv_ref_offset, s, c=color[0], edgecolors='b', label=label[0]+': sigma = '+ "{:0.6f}".format(rv_ref_sigma)+' m/s')
    #                                   '; rv: '+ "{:0.6f}".format(np.mean(rv_ref)) + ' km/s')
    plt.scatter(rv_del_hr, rv_developed_offset, s, c=color[1], edgecolors='b', label=label[1]+': sigma = '+ "{:0.6f}".format(rv_developed_sigma)+' m/s')
    #                                   '; rv: '+ "{:0.6f}".format(np.mean(rv_developed)) + ' km/s')
    
    plt.legend(loc="upper right", prop={'size':12})
    plt.xlabel('Times [hrs]')
    plt.ylabel('RV [m/s]')
    ymax = math.ceil(max(np.amax(rv_ref_offset), np.amax(rv_developed_offset)))
    ymin = math.floor(min(np.amin(rv_ref_offset), np.amin(rv_developed_offset)))
    xmin = math.ceil(np.amin(rv_del_hr))
    xmax = math.floor(np.amax(rv_del_hr))
    plt.ylim((-1.5, 1.5))
    plt.xlim((xmin-1, xmax+1))
    if title is not None:
        plt.title(title)
    
    if savefig is not None:
        plt.savefig(savefig)
    plt.show()    
    

In [ ]:
import math
def plot_velocity_time_2(ref_date, rv_infos, title=None, labels=[], colors=None):
    plt.figure(figsize=(20,10))
        
    ref_time = Time(ref_date, format='isot', scale='utc')  
    ref_jd = ref_time.jd
    
    total_rv = len(rv_infos[0])
    rv_info = rv_infos[0]
    jd_list = np.array([rv_info[i][1] for i in range(total_rv)])   
    
    rv_del_hr = (jd_list-ref_jd)*24.0
    rv_ref = np.array([rv_info[i][2] for i in range(total_rv)])
    rv_developed = np.array([rv_info[i][3] for i in range(total_rv)])
   
    rv_ref_offset = (rv_ref - np.mean(rv_ref))*1000
    rv_developed_offset = (rv_developed - np.mean(rv_developed)) * 1000
   
    rv_ref_sigma = np.std(rv_ref_offset)
    rv_developed_sigma = np.std(rv_developed_offset)
    
    plt.subplot(1, 2, 1)  
    s = 100
    
    if colors is None:
        color = ['cyan', 'yellow']
    else:
        color = colors[0]
    label = labels[0]    
    plt.scatter(rv_del_hr, rv_ref_offset, s, c=color[0], edgecolors='b', label=label[0]+': sigma = '+ "{:0.6f}".format(rv_ref_sigma)+' m/s')
    plt.scatter(rv_del_hr, rv_developed_offset, s, c=color[1], edgecolors='b', label=label[1]+': sigma = '+ "{:0.6f}".format(rv_developed_sigma)+' m/s')                                 
    
    plt.legend(loc="upper right", prop={'size':12})
    plt.xlabel('Times [hrs]')
    plt.ylabel('RV [m/s]')
    ymax = math.ceil(max(np.amax(rv_ref_offset), np.amax(rv_developed_offset)))
    ymin = math.floor(min(np.amin(rv_ref_offset), np.amin(rv_developed_offset)))
    xmin = math.ceil(np.amin(rv_del_hr))
    xmax = math.floor(np.amax(rv_del_hr))
    plt.ylim((-1.5, 1.5))
    plt.xlim((xmin-1, xmax+1))
    if title is not None:
        plt.title(title)
    
    total_rv = len(rv_infos[1])
    rv_info = rv_infos[1]
    jd_list = np.array([rv_info[i][1] for i in range(total_rv)])   
    
    rv_del_hr = (jd_list-ref_jd)*24.0
    rv_ref = np.array([rv_info[i][2] for i in range(total_rv)])
    rv_developed = np.array([rv_info[i][3] for i in range(total_rv)])
   
    rv_ref_offset = (rv_ref - np.mean(rv_ref))*1000
    rv_developed_offset = (rv_developed - np.mean(rv_developed)) * 1000
   
    rv_ref_sigma = np.std(rv_ref_offset)
    rv_developed_sigma = np.std(rv_developed_offset)
    
    plt.subplot(1, 2, 2)  
    
    if colors is None:
        color = ['cyan', 'yellow']
    else:
        color = colors[1]
    label = labels[1]    
    plt.scatter(rv_del_hr, rv_ref_offset, s, c=color[0], edgecolors='b', label=label[0]+': sigma = '+ "{:0.6f}".format(rv_ref_sigma)+' m/s')
    plt.scatter(rv_del_hr, rv_developed_offset, s, c=color[1], edgecolors='b', label=label[1]+': sigma = '+ "{:0.6f}".format(rv_developed_sigma)+' m/s')                                 
    
    plt.legend(loc="upper right", prop={'size':12})
    plt.xlabel('Times [hrs]')
    plt.ylabel('RV [m/s]')
    ymax = math.ceil(max(np.amax(rv_ref_offset), np.amax(rv_developed_offset)))
    ymin = math.floor(min(np.amin(rv_ref_offset), np.amin(rv_developed_offset)))
    xmin = math.ceil(np.amin(rv_del_hr))
    xmax = math.floor(np.amax(rv_del_hr))
    plt.ylim((-1.5, 1.5))
    plt.xlim((xmin-1, xmax+1))
    if title is not None:
        plt.title(title)
    plt.show()    

## RV computation on NEID L1 data and results from the pipeline development (OrderTrace/PolygonClipping2)

In [ ]:
#for f in range(total_file):
f_config = configparser.ConfigParser()
f_config.read(TEST_DIR+'NEIDdata/neid_stars.config')
star = 'Tau Ceti'
star_config = f_config[star]

print(star)
#import pdb; pdb.set_trace()

t0 = time.time()
config = {
    'starname': star,
    'star_rv': -20,
    'obslon': -111.600562,
    'obslat': 31.958092,
    'obsalt': 2091.0, #in meters
    'ra2000': Angle(star_config['ra']+"hours").deg,
    'de2000': Angle(star_config['dec']+"degrees").deg,
    'pm_ra': float(star_config['pmra']),  #mas/yr
    'pm_dec': float(star_config['pmdec']), #mas/yr
    'parallax': float(star_config['plx']), #mas
    'step': 0.25,
    'mask_width': 0.5, #km/s on either side of line center
    'air_to_vaccum': True
    }

# for my data
start_order=3
end_order= 82
order_diff = 7
start_x_pos = 600

spectrum_order = end_order - start_order + 1

master_dir = TEST_DIR

width_type = 'for_width_3'        # 'for_fixed_width', 'for_width_3', 'for_width_2'
method = 'optimal_not_rectified'  #'optimal_not_rectified','optimal_vertical_fraction' or 'optimal_norm_fraction'
spectrum_path = master_dir + 'order_trace_test/for_optimal_extraction/output/rv_NEID_3sigma/'+width_type+\
                                        '/NEID_*_extraction_'+method
spectrum_path_neid = master_dir + 'NEIDdata/TAUCETI_20191217/L1/'

ref_L2_path = master_dir + 'NEIDdata/TAUCETI_20191217/L2/neidL2_20191217T'
ref_L1_path = master_dir + 'NEIDdata/TAUCETI_20191217/L1/neidL1_20191217T'
mask_path = master_dir + 'rv_test/stellarmasks/G2.espresso.mas'
outfolder = master_dir + 'order_trace_test/for_radial_velocity/output/'

spectraname = glob.glob(spectrum_path + '*.fits')
spectraname_neid = glob.glob(spectrum_path_neid + '*.fits')
spectraname.sort()

total_file = len(spectraname)

radial_vel = RadialVelocity(config, spectrum_order, mask_path)
fits_list = ['023129', '023815','024240','024704',  '025129', '025613', '030057','030724','031210','031636']

rv_info = []

print(spectraname)
#import pdb;pdb.set_trace()

if radial_vel.init_calculation():
    #for f in range(0):
    for f in range(total_file):
        print('File ', f+1, 'of ', total_file, ' file: ', spectraname[f])
        
        pre_extraction = '3sigma/'+width_type+'/NEID_'
        i1 = spectraname[f].index(pre_extraction)+len(pre_extraction)
        i2 = spectraname[f].index('_extraction', i1)
        
        #i1 =  spectraname[f].index('_20191217T')+len('_20191217T')
        #i2 =  spectraname[f].index('.fits', i1)
        print(spectraname[f][i1:i2])

        code = spectraname[f][i1:i2]

        outfile_neid = outfolder + 'rv_NEID/rv_'+code+'.fits'
        outfile_cindy = outfolder + 'rv_NEID_3sigma/'+method+'/rv_cindy_'+width_type+'_'+code+'.fits'

        wavecalib_file = ref_L1_path + code + '.fits'
            
        o_spectrum, hdr = fits.getdata(spectraname[f], header=True)
        o_spectrum_neid, hdr_neid = fits.getdata(wavecalib_file, header=True)
        
        # set bad columns
        o_spectrum[:, 435:451] = np.nan
        #o_spectrum_neid[:, 435:451] = np.nan
        o_spectrum[:, 1930:1945] = np.nan
        #o_spectrum_neid[:, 1930:1945] = np.nan
        
        print('neid: ', wavecalib_file)
         
        refL1_ccf_head = fits.open(wavecalib_file)
        L1_time = refL1_ccf_head[0].header['SSBJD100']
  
        end_x = np.shape(o_spectrum)[1]-start_x_pos
  
        save_ccf = radial_vel.get_rv_on_spectrum(o_spectrum, hdr, \
                    wavelength_calib_file=wavecalib_file, hdu_in_calib = 7, \
                    start_order=start_order, end_order=end_order, \
                    start_x = start_x_pos, end_x = end_x, BC_corr=refL1_ccf_head[0].header['SSBZ100'], order_diff=7, jd_time=L1_time)
        output_ccf = radial_vel.analyze_ccf(save_ccf)      #save_ccf is the same of output_ccf
       
        save_neid_ccf = radial_vel.get_rv_on_spectrum(o_spectrum_neid, hdr, \
                    wavelength_calib_file=wavecalib_file, hdu_in_calib = 7, \
                    start_order=start_order+order_diff, end_order=end_order+order_diff, \
                    start_x = start_x_pos, end_x = end_x, BC_corr=refL1_ccf_head[0].header['SSBZ100'], order_diff=0, jd_time=L1_time)
       
        output_neid_ccf = radial_vel.analyze_ccf(save_neid_ccf)
        
        #plot_ccf(output_neid_ccf, spectrum_order, code+'_neid')
        #plot_ccf(output_ccf, spectrum_order, code+'_cindy')
        
        gaussian_fit, rv_result, g_x, g_y = radial_vel.fit_ccf(output_ccf)
        neid_fit, neid_mean, n_x, n_y = radial_vel.fit_ccf(output_neid_ccf)

        plot_gaussian_on_curve(gaussian_fit, g_x, g_y, title=spectraname[f], ref_curve=n_y, ref_gaussian=neid_fit)
        
        rv_info.append([code, L1_time, neid_fit.mean.value, gaussian_fit.mean.value])
        #rv_info.append([code, L1_time, neid_fit.mean.value, 0]) # set neid data only
        radial_vel.output_ccf_to_fits(output_ccf, outfile_cindy,  None, rv_result)
        radial_vel.output_ccf_to_fits(output_neid_ccf, outfile_neid, None, neid_mean)
            
t3 = time.time()

## RV vs. time from the results of above RV computation

In [ ]:
neid_color = 'yellow'
cindy_color = 'cyan'
plot_velocity_time('2019-12-17T00:00:00', rv_info, label=['neid', 'developed'], color=[neid_color, cindy_color])  

## For test: rv vs. time on two set of ccf files side by side (from NEID and pipeline development). Each chart shows rv vs. time by removing the rv value of one order from a specific ccf file.

In [ ]:
fits_list = ['023129', '023815','024240','024704',  '025129', '025613', '030057','030724','031210','031636']
cindy_dir = TEST_DIR+'order_trace_test/for_radial_velocity/output/rv_NEID_3sigma/v0'
neid_dir = TEST_DIR+'order_trace_test/for_radial_velocity/output/rv_NEID/v0'
L1_data = TEST_DIR+'NEIDdata/TAUCETI_20191217/L1/neidL1_20191217T'
neid_color = 'yellow'
cindy_color = 'cyan'
test_color = 'red'

cindy_ccfs = glob.glob(cindy_dir + '/*.fits')
neid_ccfs = glob.glob(neid_dir + '/*.fits')
cindy_ccfs.sort()
neid_ccfs.sort()
rv_info = []
select_idx = 3
select_neid_ccf = None
select_cindy_ccf = None
for i in range(len(fits_list)):
    if fits_list[i] in neid_ccfs[i] and fits_list[i] in cindy_ccfs[i]:
        n_ccf, n_hdr = fits.getdata(neid_ccfs[i], header=True)
        c_ccf, c_hdr = fits.getdata(cindy_ccfs[i], header=True)
        
        if i == select_idx:
            select_neid_ccf = n_ccf
            select_cindy_ccf = c_ccf
        neid_fit, rv_neid, n_x, n_y = radial_vel.fit_ccf(n_ccf)
        cindy_fit, rv_cindy, c_x, c_y = radial_vel.fit_ccf(c_ccf)
        L1_fits = L1_data + fits_list[i] + '.fits'
        L1_hdr = fits.open(L1_fits)
        L1_time = L1_hdr[0].header['SSBJD100']
        rv_info.append([fits_list[i], L1_time, neid_fit.mean.value, cindy_fit.mean.value])
    else:
        print(fits_list[i], ' is missing in ', neid_ccfs[i], ' or ', cindy_ccfs[i])

plot_velocity_time('2019-12-17T00:00:00', rv_info, label=['neid', 'cindy'], color=[neid_color, cindy_color])

fits_check = fits_list[1]
total_order = np.shape(select_neid_ccf)[0] - 3

print("   remove one order of ccf values ccf file on ", fits_list[select_idx], " " ,\
                          str(select_idx+1)+'/'+str(total_order))        
for i in range(1, total_order):
    print('order ', i)
    new_rv_neid_info = []
    new_rv_cindy_info = []
    for j in range(len(fits_list)):
        new_rv_neid_info.append(rv_info[j].copy())
        new_rv_neid_info[j][3] = rv_info[j][2]    # copy neid data to the test position
        new_rv_cindy_info.append(rv_info[j].copy())
        new_rv_cindy_info[j][2] = new_rv_cindy_info[j][3]
        
    new_neid_select_ccf = select_neid_ccf.copy()
    new_cindy_select_ccf = select_cindy_ccf.copy()
    # do not count the select order for ccf 
    new_neid_select_ccf[total_order+2, :] = select_neid_ccf[total_order+2, :] - select_neid_ccf[i, :]
    # double check if total_order+2 are correct
           
    new_fit, new_rv, new_x, new_y = radial_vel.fit_ccf(new_neid_select_ccf)
    new_rv_neid_info[select_idx][3] = new_rv
    
    new_cindy_select_ccf[total_order+2, :] = select_cindy_ccf[total_order+2, :] - select_cindy_ccf[i, :]
    new_fit, new_rv, new_x, new_y = radial_vel.fit_ccf(new_cindy_select_ccf)
    new_rv_cindy_info[select_idx][3] = new_rv
    
    plot_velocity_time_2('2019-12-17T00:00:00', [new_rv_neid_info, new_rv_cindy_info],  
                       labels=[['neid', 'not count '+str(i)], ['cindy', 'not count '+str(i)]],
                       colors=[[neid_color, test_color], [cindy_color, test_color]])
      

## For test: rv vs. time plot on one set of ccf files (from NEID and pipeline development sequentially). Each chart shows rv vs. time by removing the rv value of one order from a specific ccf file.

In [ ]:
fits_list = ['023129', '023815','024240','024704',  '025129', '025613', '030057','030724','031210','031636']
cindy_dir = TEST_DIR+'order_trace_test/for_radial_velocity/output/rv_NEID_3sigma'
neid_dir = TEST_DIR+'order_trace_test/for_radial_velocity/output/rv_NEID'
L1_data = TEST_DIR+'NEIDdata/TAUCETI_20191217/L1/neidL1_20191217T'
neid_color = 'yellow'
cindy_color = 'cyan'
test_color = 'red'

output_dir = TEST_DIR+'order_trace_test/for_radial_velocity/output/ccf_on_order_removal/'

cindy_ccfs = glob.glob(cindy_dir + '/*.fits')
neid_ccfs = glob.glob(neid_dir + '/*.fits')
cindy_ccfs.sort()
neid_ccfs.sort()
neid_means = np.zeros(len(fits_list))
cindy_means = np.zeros(len(fits_list))
rv_info = []
select_idx = 6
select_neid_ccf = None
select_cindy_ccf = None
for i in range(len(fits_list)):
    if fits_list[i] in neid_ccfs[i] and fits_list[i] in cindy_ccfs[i]:
        n_ccf, n_hdr = fits.getdata(neid_ccfs[i], header=True)
        c_ccf, c_hdr = fits.getdata(cindy_ccfs[i], header=True)
        
        if i == select_idx:
            import pdb;pdb.set_trace()
            select_neid_ccf = n_ccf
            select_cindy_ccf = c_ccf
        neid_fit, rv_neid, n_x, n_y = radial_vel.fit_ccf(n_ccf)
        cindy_fit, rv_cindy, c_x, c_y = radial_vel.fit_ccf(c_ccf)
        neid_means[i] = rv_neid
        cindy_means[i] = rv_cindy
        L1_fits = L1_data + fits_list[i] + '.fits'
        L1_hdr = fits.open(L1_fits)
        L1_time = L1_hdr[0].header['SSBJD100']
        rv_info.append([fits_list[i], L1_time, neid_fit.mean.value, cindy_fit.mean.value])
    else:
        print(fits_list[i], ' is missing in ', neid_ccfs[i], ' or ', cindy_ccfs[i])

plot_velocity_time('2019-12-17T00:00:00', rv_info, label=['neid', 'cindy'], color=[neid_color, cindy_color])

fits_check = fits_list[1]
total_order = np.shape(select_neid_ccf)[0] - 3

print("   remove one order of ccf values from neid ccf file on ", fits_list[select_idx], " " ,\
                          str(select_idx+1)+'/'+str(total_order))        
import pdb;pdb.set_trace()
for i in range(1, total_order):
    new_rv_info = []
    for j in range(len(fits_list)):
        new_rv_info.append(rv_info[j].copy())
        new_rv_info[j][3] = rv_info[j][2]    # copy neid data to the test position
    new_neid_select_ccf = select_neid_ccf.copy()
    # do not count the select order for ccf 
    new_neid_select_ccf[total_order+2, :] = select_neid_ccf[total_order+2, :] - select_neid_ccf[i, :]
    new_fit, new_rv, new_x, new_y = radial_vel.fit_ccf(new_neid_select_ccf)
    new_rv_info[select_idx][3] = new_rv
    
    
    plot_velocity_time('2019-12-17T00:00:00', new_rv_info, \
                       label=['neid', 'not count '+str(i)], color=[neid_color, test_color], 
                       savefig=output_dir+'neid_'+fits_list[select_idx]+'_not_include_'+str(i)+'.png')
      
print("   remove one order of ccf values from cindy's ccf file on ", fits_list[select_idx], " " ,\
                          str(select_idx+1)+'/'+str(total_order))     

import pdb;pdb.set_trace()
for i in range(1, total_order):
    new_rv_info = []
    for j in range(len(fits_list)):
        new_rv_info.append(rv_info[j].copy())
        new_rv_info[j][2] = new_rv_info[j][3]
        
    new_cindy_select_ccf = select_cindy_ccf.copy()
    new_cindy_select_ccf[total_order+2, :] = select_cindy_ccf[total_order+2, :] - select_cindy_ccf[i, :]
    new_fit, new_rv, new_x, new_y = radial_vel.fit_ccf(new_cindy_select_ccf)
    new_rv_info[select_idx][3] = new_rv
    
    plot_velocity_time('2019-12-17T00:00:00', new_rv_info, \
                       label=['cindy', 'not count '+str(i)], color=[cindy_color, test_color], 
                       savefig=output_dir+'cindy_'+fits_list[select_idx]+'_not_include_'+str(i)+'.png')
      


## For test: ccf plot on a set of specific files

### ccf of every single order in one plot

In [ ]:
def plot_ccf(ccf, total_order, filecode, plot_per_row = 4, savefig=None ):
    next_order = 0
    row = 0
    
    total_row = total_order//plot_per_row if total_order%plot_per_row == 0 else (total_order+plot_per_row)//plot_per_order
    fig, ax = plt.subplots(total_row, plot_per_row, figsize=(16,40))
    #fig.suptitle(filecode, fontsize=12)
    #plt.subplots_adjust(hspace=0.2)
    for next_order in range(1, total_order, plot_per_row):
        for i in range(next_order, next_order+plot_per_row):
            if i >= total_order:
                ax[row, i-next_order].plot(ccf[total_order+1, :], ccf[total_order+2, :])
                ax[row, i-next_order].text(ccf[total_order+1, 20], ccf[total_order+2, 20], 'all')
                break
            ax[row, i-next_order].plot(ccf[total_order+1, :], ccf[i, :])
            ax[row, i-next_order].text(ccf[total_order+1, 20], ccf[i, 20], str(i))
        row += 1
    if savefig is not None:
        plt.savefig(savefig)
    plt.show()              
    

### ccf of all orders in one plot

In [ ]:
def plot_ccf_in_one(ccf, total_order, filecode, savefig=None):
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'b', '#f5a623', '#9013fe', '#8b572a']
    
    c_size = len(colors)
    plt.figure(figsize=(20,20))
    x_size = np.shape(ccf)[1]
    for next_order in range(1, total_order, c_size):
        for i in range(next_order, next_order+c_size):
            plt.plot(ccf[total_order+1, :], ccf[i, :], c=colors[i-next_order])
            loc = (i*4)%x_size
            plt.annotate(i, (ccf[total_order+1, loc], ccf[i, loc]), fontsize=14)
    plt.title(filecode, fontsize=16)
    if savefig is not None:
        plt.savefig(savefig)
    plt.show()    

In [ ]:
fits_list = ['023129', '023815','024240','024704',  '025129', '025613', '030057','030724','031210','031636']
cindy_dir = TEST_DIR+'order_trace_test/for_radial_velocity/output/rv_NEID_3sigma'
neid_dir = TEST_DIR+'order_trace_test/for_radial_velocity/output/rv_NEID'
L1_data = TEST_DIR+'NEIDdata/TAUCETI_20191217/L1/neidL1_20191217T'

output_plot = TEST_DIR+'order_trace_test/for_radial_velocity/output/'
cindy_ccfs = glob.glob(cindy_dir + '/*.fits')
neid_ccfs = glob.glob(neid_dir + '/*.fits')
cindy_ccfs.sort()
neid_ccfs.sort()

file_set = list(range(len(fits_list)))

neid_color = 'yellow'
cindy_color = 'cyan'
rv_info = []
for i in range(len(fits_list)):
    if fits_list[i] in neid_ccfs[i] and fits_list[i] in cindy_ccfs[i]:
        n_ccf, n_hdr = fits.getdata(neid_ccfs[i], header=True)
        c_ccf, c_hdr = fits.getdata(cindy_ccfs[i], header=True)
        
        neid_fit, rv_neid, n_x, n_y = radial_vel.fit_ccf(n_ccf)
        cindy_fit, rv_cindy, c_x, c_y = radial_vel.fit_ccf(c_ccf)

        L1_fits = L1_data + fits_list[i] + '.fits'
        L1_hdr = fits.open(L1_fits)
        L1_time = L1_hdr[0].header['SSBJD100']
        rv_info.append([fits_list[i], L1_time, rv_neid, rv_cindy])
    else:
        print(fits_list[i], ' is missing in ', neid_ccfs[i], ' or ', cindy_ccfs[i])

plot_velocity_time('2019-12-17T00:00:00', rv_info, label=['neid', 'cindy'], color=[neid_color, cindy_color])


for i in file_set:
    print('**** '+ str(i+1)+ '. ccf neid on '+fits_list[i])
    filename = neid_ccfs[i]
    ccf_data, hdr = fits.getdata(filename, header=True)
    spectrum_order = np.shape(ccf_data)[0] - 3
    plot_ccf(ccf_data, spectrum_order, 'neid'+fits_list[i], 4, savefig=output_plot+'one_ccf_neid_on '+fits_list[i]+'.png')
    #plot_ccf_in_one(ccf_data, spectrum_order, 'neid '+fits_list[i], savefig=output_plot+'ccf_neid_on '+fits_list[i]+'.png')
    
for i in file_set:
    print('**** '+ str(i+1)+ '. ccf cindy on '+fits_list[i])
    filename = cindy_ccfs[i]
    ccf_data, hdr = fits.getdata(filename, header=True)
    spectrum_order = np.shape(ccf_data)[0] - 3
    plot_ccf(ccf_data, spectrum_order, 'cindy'+fits_list[i], 4, savefig=output_plot+'one_ccf_cindy_on '+fits_list[i]+'.png')
    #plot_ccf_in_one(ccf_data, spectrum_order, 'cindy '+fits_list[i], savefig=output_plot+'ccf_cindy_on '+fits_list[i]+'.png')

## For test: ccf (rv vs. velocities) results from NEID L2 files

In [ ]:
FIT_G = fitting.LevMarLSQFitter()
L2_dir = TEST_DIR+'NEIDdata/TAUCETI_20191217/L2/neidL2_20191217T'
L2_files = glob.glob(L2_dir + '*.fits')
velocity_cut=100.0
rv_guess = -20
velocities = np.arange(-82, 82) * 0.25 + rv_guess

fits_list = ['023129', '023815','024240','024704', '030057','030724','031210','031636', '025129', '025613']
rv_info = []
for f in fits_list:
    print(f)
    L2_hdu = fits.open(L2_dir+f+'.fits')
    ccf = L2_hdu[12].data
    row, col = np.shape(ccf)
    print('total row ', np.shape(ccf))
    sum_ccf = np.sum(ccf[0:row, :], axis=0)
    print('sum ccf shape: ', np.shape(sum_ccf))
                 
    g_init = models.Gaussian1D(amplitude=-1e7, mean=rv_guess, stddev=5.0)
    i_cut = (velocities >= rv_guess - velocity_cut) & (velocities <= rv_guess + velocity_cut)
    g_x = velocities[i_cut]
    g_y = sum_ccf[i_cut]-np.nanmedian(sum_ccf)
    gaussian_fit = FIT_G(g_init, g_x, g_y)
 
    plot_gaussian_on_curve(gaussian_fit, g_x, g_y, title=f)         
    L1_time = L2_hdu[0].header['SSBJD100']
    rv_info.append([f, L1_time, gaussian_fit.mean.value, 0.0])
    